## Imports

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pickle
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

## Clean data

In [2]:
clinical_data = pd.read_table('D:\\MyProjects\\Cancer detection\\data review\\brca_metabric\\data_clinical_patient.txt')
clinical_data.head()

,#Patient Identifier,Lymph nodes examined positive,Nottingham prognostic index,Cellularity,Chemotherapy,Cohort,ER status measured by IHC,HER2 status measured by SNP6,Hormone Therapy,Inferred Menopausal State,...,Overall Survival Status,Pam50 + Claudin-low subtype,3-Gene classifier subtype,Patient's Vital Status,Primary Tumor Laterality,Radio Therapy,Tumor Other Histologic Subtype,Type of Breast Surgery,Relapse Free Status,Relapse Free Status (Months)
0,#Identifier to uniquely specify a patient.,Number of lymphnodes positive,Nottingham prognostic index,Tumor Content,Chemotherapy.,Cohort.,ER status measured by IHC,HER2 status measured by SNP6,Hormone Therapy,Inferred Menopausal State,...,Overall patient survival status.,Pam50 + Claudin-low subtype,3-Gene classifier subtype,The survival state of the person.,"For tumors in paired organs, designates the si...",Radio Therapy,Text to describe a tumor's histologic subtype ...,Type of Breast Surgery,Relapse Free Status (Recurred = cases where th...,Time until last follow-up or relapse (in months)
1,#STRING,STRING,NUMBER,STRING,STRING,STRING,STRING,STRING,STRING,STRING,...,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,NUMBER
2,#1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,PATIENT_ID,LYMPH_NODES_EXAMINED_POSITIVE,NPI,CELLULARITY,CHEMOTHERAPY,COHORT,ER_IHC,HER2_SNP6,HORMONE_THERAPY,INFERRED_MENOPAUSAL_STATE,...,OS_STATUS,CLAUDIN_SUBTYPE,THREEGENE,VITAL_STATUS,LATERALITY,RADIO_THERAPY,HISTOLOGICAL_SUBTYPE,BREAST_SURGERY,RFS_STATUS,RFS_MONTHS
4,MB-0000,10,6.044,NaN,NO,1,Positve,NEUTRAL,YES,Post,...,0:LIVING,claudin-low,ER-/HER2-,Living,Right,YES,Ductal/NST,MASTECTOMY,0:Not Recurred,138.65


In [3]:
clinical_data.drop([0,1,2,3], axis=0,inplace=True)
new_idx = clinical_data[['#Patient Identifier']]
new_idx = new_idx.rename({'#Patient Identifier':'Identifier'}, axis=1)
clinical_data.drop(['#Patient Identifier'], axis=1, inplace=True)
clinical_data.index = new_idx['Identifier']
clinical_data['Lymph nodes examined positive'] = clinical_data['Lymph nodes examined positive'].apply(lambda x: float(x))
clinical_data.head()

,Lymph nodes examined positive,Nottingham prognostic index,Cellularity,Chemotherapy,Cohort,ER status measured by IHC,HER2 status measured by SNP6,Hormone Therapy,Inferred Menopausal State,Sex,...,Overall Survival Status,Pam50 + Claudin-low subtype,3-Gene classifier subtype,Patient's Vital Status,Primary Tumor Laterality,Radio Therapy,Tumor Other Histologic Subtype,Type of Breast Surgery,Relapse Free Status,Relapse Free Status (Months)
Identifier,,,,,,,,,,,,,,,,,,,,,
MB-0000,10.0,6.044,NaN,NO,1,Positve,NEUTRAL,YES,Post,Female,...,0:LIVING,claudin-low,ER-/HER2-,Living,Right,YES,Ductal/NST,MASTECTOMY,0:Not Recurred,138.65
MB-0002,0.0,4.02,High,NO,1,Positve,NEUTRAL,YES,Pre,Female,...,0:LIVING,LumA,ER+/HER2- High Prolif,Living,Right,YES,Ductal/NST,BREAST CONSERVING,0:Not Recurred,83.52
MB-0005,1.0,4.03,High,YES,1,Positve,NEUTRAL,YES,Pre,Female,...,1:DECEASED,LumB,NaN,Died of Disease,Right,NO,Ductal/NST,MASTECTOMY,1:Recurred,151.28
MB-0006,3.0,4.05,Moderate,YES,1,Positve,NEUTRAL,YES,Pre,Female,...,0:LIVING,LumB,NaN,Living,Right,YES,Mixed,MASTECTOMY,0:Not Recurred,162.76
MB-0008,8.0,6.08,High,YES,1,Positve,NEUTRAL,YES,Post,Female,...,1:DECEASED,LumB,ER+/HER2- High Prolif,Died of Disease,Right,YES,Mixed,MASTECTOMY,1:Recurred,18.55


In [4]:
gene_data = pd.read_table('D:\\MyProjects\\Cancer detection\\data review\\brca_metabric\\data_mrna_agilent_microarray.txt')
gene_data.head()

,Hugo_Symbol,Entrez_Gene_Id,MB-0362,MB-0346,MB-0386,MB-0574,MB-0503,MB-0641,MB-0201,MB-0218,...,MB-6122,MB-6192,MB-4820,MB-5527,MB-5167,MB-5465,MB-5453,MB-5471,MB-5127,MB-4313
0,RERE,473.0,8.676978,9.653589,9.033589,8.814855,9.274265,9.286585,8.437347,8.569973,...,8.756024,8.804947,9.991215,9.595923,9.637249,8.131637,9.606915,9.049296,8.858622,8.415867
1,RNF165,494470.0,6.075331,6.687887,5.910885,5.628740,5.908698,6.206729,6.095592,6.383530,...,7.101087,5.601472,7.103160,6.418987,6.203370,9.101942,7.427494,6.850000,6.550450,6.831722
2,CD049690,NaN,5.453928,5.454185,5.501577,5.471941,5.531743,5.372668,5.693519,5.401276,...,5.344674,5.364348,5.504866,5.277314,5.439207,5.423027,5.534115,5.339346,5.566071,5.541395
3,BC033982,NaN,4.994525,5.346010,5.247467,5.316523,5.244094,5.167365,5.189106,5.416517,...,5.300466,4.861101,4.980904,5.436855,5.346776,4.939292,5.062191,5.166765,5.140141,5.266802
4,PHF7,51533.0,5.838270,5.600876,6.030718,5.849428,5.964661,5.783374,5.737572,5.923928,...,5.653481,5.922894,6.181493,5.992153,6.230316,5.644587,5.927409,6.117095,5.936371,7.408960


In [5]:
gene_data.drop("Entrez_Gene_Id", axis=1, inplace=True)
new_idx = gene_data[['Hugo_Symbol']]
new_idx = new_idx.rename({'Hugo_Symbol':'Identifier'}, axis=1)
gene_data.drop(['Hugo_Symbol'], axis=1, inplace=True)
gene_data.index = new_idx['Identifier']
gene_data = gene_data.transpose()
gene_data.head()

Identifier,RERE,RNF165,CD049690,BC033982,PHF7,CIDEA,PAPD4,AI082173,SLC17A3,SDS,...,BX115874,BX107598,UGCGL1,VPS72,CSMD3,CC2D1A,CB986545,IGSF9,DA110839,FAM71A
MB-0362,8.676978,6.075331,5.453928,4.994525,5.838270,6.397503,7.906217,5.259461,5.702379,6.930741,...,5.271343,5.680321,7.688492,8.084979,5.161796,6.353215,4.836483,7.304643,5.251843,5.049591
MB-0346,9.653589,6.687887,5.454185,5.346010,5.600876,5.246319,8.267256,5.380069,5.521794,6.141689,...,5.942887,5.461069,7.804165,8.349115,5.197392,6.132355,5.316819,7.933324,5.450611,5.316790
MB-0386,9.033589,5.910885,5.501577,5.247467,6.030718,10.111816,7.959291,5.262024,5.689533,6.529312,...,5.174498,5.304030,7.934309,8.406332,8.087722,6.366335,5.466419,7.580336,5.235394,5.461617
MB-0574,8.814855,5.628740,5.471941,5.316523,5.849428,6.116868,9.206376,5.396576,5.439130,6.430102,...,5.116749,5.632249,7.744562,8.310019,5.780062,6.424048,5.193150,6.903654,5.091927,5.227130
MB-0503,9.274265,5.908698,5.531743,5.244094,5.964661,7.828171,8.706646,5.167213,5.417484,6.684893,...,5.402314,5.472185,7.701394,8.137014,5.498185,6.214301,5.274600,6.839417,5.315224,5.027476


In [6]:
genes_set = set(gene_data.index)
clinical_set = set(clinical_data.index)
to_del_from_clinical = list(clinical_set - genes_set)
clinical_data.drop(to_del_from_clinical, axis=0, inplace=True)

In [7]:
clinical_to_drop = ['Nottingham prognostic index',
       'Cellularity', 'Chemotherapy', 'Cohort', 'ER status measured by IHC',
       'HER2 status measured by SNP6', 'Hormone Therapy',
       'Inferred Menopausal State', 'Sex', 'Integrative Cluster',
       'Age at Diagnosis', 'Overall Survival (Months)',
       'Overall Survival Status','3-Gene classifier subtype', "Patient's Vital Status",
       'Primary Tumor Laterality', 'Radio Therapy',
       'Tumor Other Histologic Subtype', 'Type of Breast Surgery',
       'Relapse Free Status', 'Relapse Free Status (Months)']
clinical_data_useful_only = clinical_data.drop(clinical_to_drop, axis=1)
clinical_data_useful_only.head()

,Lymph nodes examined positive,Pam50 + Claudin-low subtype
Identifier,,
MB-0000,10.0,claudin-low
MB-0002,0.0,LumA
MB-0005,1.0,LumB
MB-0006,3.0,LumB
MB-0008,8.0,LumB


In [8]:
clinical_data.shape, gene_data.shape

((1904, 23), (1904, 24368))

In [9]:
data = gene_data.join(clinical_data_useful_only) 
data['Pam50 + Claudin-low subtype'].unique()

array(['LumA', 'Her2', 'claudin-low', 'Basal', 'LumB', 'Normal', 'NC'],
      dtype=object)

In [10]:
normal_idx = list(data[data['Pam50 + Claudin-low subtype'] == 'Normal'].index)
NC_idx = list(data[data['Pam50 + Claudin-low subtype'] == 'NC'].index)
claudin_low_idx = list(data[data['Pam50 + Claudin-low subtype'] == 'claudin-low'].index)
indices_to_drop = normal_idx + NC_idx + claudin_low_idx
data.drop(indices_to_drop, axis=0, inplace=True)
data.head()

,RERE,RNF165,CD049690,BC033982,PHF7,CIDEA,PAPD4,AI082173,SLC17A3,SDS,...,UGCGL1,VPS72,CSMD3,CC2D1A,CB986545,IGSF9,DA110839,FAM71A,Lymph nodes examined positive,Pam50 + Claudin-low subtype
MB-0362,8.676978,6.075331,5.453928,4.994525,5.838270,6.397503,7.906217,5.259461,5.702379,6.930741,...,7.688492,8.084979,5.161796,6.353215,4.836483,7.304643,5.251843,5.049591,1.0,LumA
MB-0346,9.653589,6.687887,5.454185,5.346010,5.600876,5.246319,8.267256,5.380069,5.521794,6.141689,...,7.804165,8.349115,5.197392,6.132355,5.316819,7.933324,5.450611,5.316790,5.0,Her2
MB-0386,9.033589,5.910885,5.501577,5.247467,6.030718,10.111816,7.959291,5.262024,5.689533,6.529312,...,7.934309,8.406332,8.087722,6.366335,5.466419,7.580336,5.235394,5.461617,8.0,LumA
MB-0574,8.814855,5.628740,5.471941,5.316523,5.849428,6.116868,9.206376,5.396576,5.439130,6.430102,...,7.744562,8.310019,5.780062,6.424048,5.193150,6.903654,5.091927,5.227130,1.0,LumA
MB-0503,9.274265,5.908698,5.531743,5.244094,5.964661,7.828171,8.706646,5.167213,5.417484,6.684893,...,7.701394,8.137014,5.498185,6.214301,5.274600,6.839417,5.315224,5.027476,0.0,LumA


In [11]:
data = data.dropna()
data.isnull().values.any()

False

In [12]:
y1 = data.loc[:,'Pam50 + Claudin-low subtype']
le = LabelEncoder()
y = le.fit_transform(y1)

labels = le.classes_

X = data.drop("Pam50 + Claudin-low subtype", axis=1)
y = pd.DataFrame(y)
y.index = X.index
y.columns = ['Subtype']
y.head()

,Subtype
MB-0362,2
MB-0346,1
MB-0386,2
MB-0574,2
MB-0503,2


## Split data

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state = 0, stratify=y)

In [15]:
X_train.shape, X_test.shape

((1395, 24369), (156, 24369))

In [17]:
y_train.value_counts()

Subtype
2          606
3          414
1          196
0          179
dtype: int64

In [20]:
labels

array(['Basal', 'Her2', 'LumA', 'LumB'], dtype=object)

## Scale data

In [18]:
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

## Save data

In [19]:
with open('Saved data/X_train.pickle', 'wb') as output:
    pickle.dump(X_train, output)
    
with open('Saved data/X_test.pickle', 'wb') as output:
    pickle.dump(X_test, output)

with open('Saved data/X_train_scaled.pickle', 'wb') as output:
    pickle.dump(X_train_scaled, output)
    
with open('Saved data/X_test_scaled.pickle', 'wb') as output:
    pickle.dump(X_test_scaled, output)
        
with open('Saved data/y_train.pickle', 'wb') as output:
    pickle.dump(y_train, output)
    
with open('Saved data/y_test.pickle', 'wb') as output:
    pickle.dump(y_test, output)
    
with open('Saved data/labels.pickle', 'wb') as output:
    pickle.dump(labels, output)
    
with open('Saved models & utils/standardScaler.pickle', 'wb') as output:
    pickle.dump(ss, output)